In [1]:
import os
import numpy as np
import pandas as pd

In [2]:
def read_excel(path):
    return pd.read_excel(path)

def delete_outlier(df, HARI, JAM):
    df = df[(df['HARI'] == HARI) & (df['JAM'] == JAM)]    
    
    # Menghitung Q1 (kuartil pertama) dan Q3 (kuartil ketiga)
    Q1 = df['SMP'].quantile(0.25)
    Q3 = df['SMP'].quantile(0.75)
    IQR = Q3 - Q1
    
    # Menentukan batas bawah dan batas atas untuk outlier
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    
    # Menemukan dan menghapus outlier
    df_outlier = df[(df['SMP'] < lower_bound) | (df['SMP'] > upper_bound)]
    df_cleaned = df[(df['SMP'] >= lower_bound) & (df['SMP'] <= upper_bound)]
    
    # if(len(df_outlier)>0):
    #     print('Ada Outlier')

    # df_grouped = df_cleaned[['HARI', 'JAM', 'SMP']]
    # df_grouped = round(df_grouped.groupby(['HARI', 'JAM']).mean()).reset_index()

    return df_outlier, df_cleaned


In [3]:
path_1 = '../Data/Preprocessing/Data Losarang/2023/2_input_losarang_2023.xlsx'

df = read_excel(path_1)
df.head()

,WAKTU,TAHUN,BULAN,PEKAN,TANGGAL,HARI,JAM,MOTOR,MOBIL,TRUK/BUS,JUMLAH,SMP_MTR,SMP_MBL,SMP_TRK,SMP
0,2023-01-01 00:00:00,2023,1,1,1,Sunday,00:00,85,24,4,113,68.0,24,10.0,102
1,2023-01-01 00:15:00,2023,1,1,1,Sunday,00:15,92,19,6,117,73.6,19,15.0,108
2,2023-01-01 00:30:00,2023,1,1,1,Sunday,00:30,78,16,6,100,62.4,16,15.0,93
3,2023-01-01 00:45:00,2023,1,1,1,Sunday,00:45,78,23,4,105,62.4,23,10.0,95
4,2023-01-01 01:00:00,2023,1,1,1,Sunday,01:00,76,14,6,96,60.8,14,15.0,90


In [4]:
df_waktu = df[['HARI','JAM']]
df_waktu = df_waktu.groupby(['HARI', 'JAM']).mean().round().reset_index()
df_waktu.head()

,HARI,JAM
0,Friday,00:00
1,Friday,00:15
2,Friday,00:30
3,Friday,00:45
4,Friday,01:00


In [5]:
all_df_outlier = []
all_df_cleaned = []

for i, row in df_waktu.iterrows():
    df_outlier,  df_cleaned = delete_outlier(df, row['HARI'], row['JAM'])
    all_df_outlier.append(df_outlier)
    all_df_cleaned.append(df_cleaned)

# gabungkan data
df_outlier_combined = pd.concat(all_df_outlier)
df_cleaned_combined = pd.concat(all_df_cleaned)

In [6]:
df_outlier_combined.head()

,WAKTU,TAHUN,BULAN,PEKAN,TANGGAL,HARI,JAM,MOTOR,MOBIL,TRUK/BUS,JUMLAH,SMP_MTR,SMP_MBL,SMP_TRK,SMP
10560,2023-04-21,2023,4,3,21,Friday,00:00,1276,151,23,1450,1020.8,151,57.5,1229
11232,2023-04-28,2023,4,4,28,Friday,00:00,93,277,130,500,74.4,277,325.0,676
21312,2023-08-11,2023,8,2,11,Friday,00:00,73,100,196,369,58.4,100,490.0,648
21984,2023-08-18,2023,8,3,18,Friday,00:00,52,30,27,109,41.6,30,67.5,139
26016,2023-09-29,2023,9,5,29,Friday,00:00,9,10,18,37,7.2,10,45.0,62


In [7]:
df_cleaned_combined.head()

,WAKTU,TAHUN,BULAN,PEKAN,TANGGAL,HARI,JAM,MOTOR,MOBIL,TRUK/BUS,JUMLAH,SMP_MTR,SMP_MBL,SMP_TRK,SMP
480,2023-01-06,2023,1,1,6,Friday,00:00,28,45,43,116,22.4,45,107.5,175
1152,2023-01-13,2023,1,2,13,Friday,00:00,29,47,35,111,23.2,47,87.5,158
1824,2023-01-20,2023,1,3,20,Friday,00:00,23,44,39,106,18.4,44,97.5,160
2496,2023-01-27,2023,1,4,27,Friday,00:00,46,48,89,183,36.8,48,222.5,307
3168,2023-02-03,2023,2,1,3,Friday,00:00,46,48,89,183,36.8,48,222.5,307


In [8]:
df_outlier_combined = df_outlier_combined.sort_values(by='WAKTU').reset_index(drop=True)
df_cleaned_combined = df_cleaned_combined.sort_values(by='WAKTU').reset_index(drop=True)

In [9]:
df_cleaned_combined.to_excel('../Data/Preprocessing/Data Losarang/2023/3_clean_losarang_2023.xlsx', index=False)
df_outlier_combined.to_excel('../Data/Preprocessing/Data Losarang/2023/3_outliers_losarang_2023.xlsx', index=False)

In [10]:
len(df_cleaned_combined) + len(df_outlier_combined)

35040

In [11]:
df_cleaned_combined.head()

,WAKTU,TAHUN,BULAN,PEKAN,TANGGAL,HARI,JAM,MOTOR,MOBIL,TRUK/BUS,JUMLAH,SMP_MTR,SMP_MBL,SMP_TRK,SMP
0,2023-01-01 08:45:00,2023,1,1,1,Sunday,08:45,216,82,44,342,172.8,82,110.0,365
1,2023-01-01 09:15:00,2023,1,1,1,Sunday,09:15,192,113,38,343,153.6,113,95.0,362
2,2023-01-01 10:45:00,2023,1,1,1,Sunday,10:45,202,108,53,363,161.6,108,132.5,402
3,2023-01-01 11:00:00,2023,1,1,1,Sunday,11:00,236,107,35,378,188.8,107,87.5,383
4,2023-01-01 11:15:00,2023,1,1,1,Sunday,11:15,192,102,36,330,153.6,102,90.0,346


In [12]:
df_smp = df_cleaned_combined[['WAKTU','TAHUN', 'BULAN', 'TANGGAL', 'HARI','JAM','MOTOR','MOBIL','TRUK/BUS',
                              'SMP_MTR','SMP_MBL','SMP_TRK','SMP']]

In [13]:
df_smp.head(8)

,WAKTU,TAHUN,BULAN,TANGGAL,HARI,JAM,MOTOR,MOBIL,TRUK/BUS,SMP_MTR,SMP_MBL,SMP_TRK,SMP
0,2023-01-01 08:45:00,2023,1,1,Sunday,08:45,216,82,44,172.8,82,110.0,365
1,2023-01-01 09:15:00,2023,1,1,Sunday,09:15,192,113,38,153.6,113,95.0,362
2,2023-01-01 10:45:00,2023,1,1,Sunday,10:45,202,108,53,161.6,108,132.5,402
3,2023-01-01 11:00:00,2023,1,1,Sunday,11:00,236,107,35,188.8,107,87.5,383
4,2023-01-01 11:15:00,2023,1,1,Sunday,11:15,192,102,36,153.6,102,90.0,346
5,2023-01-01 12:00:00,2023,1,1,Sunday,12:00,191,132,39,152.8,132,97.5,382
6,2023-01-01 15:00:00,2023,1,1,Sunday,15:00,255,76,35,204.0,76,87.5,368
7,2023-01-01 15:45:00,2023,1,1,Sunday,15:45,257,71,35,205.6,71,87.5,364


In [14]:
# Round down the WAKTU column to the nearest hour
df_smp_jam = df_smp.copy()
df_smp_jam['JAM'] = df_smp['WAKTU'].dt.floor('H')

# Drop the original WAKTU column if it's not needed anymore
df_smp_jam = df_smp_jam.drop(columns=['WAKTU'])

# Format the 'JAM' column to show only hour and minute
df_smp_jam['JAM'] = df_smp_jam['JAM'].dt.strftime('%H:%M')

# Group by the rounded-down hour and sum the SMP column
df_smp_jam = df_smp_jam.groupby(['TAHUN', 'BULAN', 'TANGGAL', 'HARI', 'JAM'], as_index=False).mean().round()

In [15]:
df_smp_jam.head(8)

,TAHUN,BULAN,TANGGAL,HARI,JAM,MOTOR,MOBIL,TRUK/BUS,SMP_MTR,SMP_MBL,SMP_TRK,SMP
0,2023,1,1,Sunday,08:00,216.0,82.0,44.0,173.0,82.0,110.0,365.0
1,2023,1,1,Sunday,09:00,192.0,113.0,38.0,154.0,113.0,95.0,362.0
2,2023,1,1,Sunday,10:00,202.0,108.0,53.0,162.0,108.0,132.0,402.0
3,2023,1,1,Sunday,11:00,214.0,104.0,36.0,171.0,104.0,89.0,364.0
4,2023,1,1,Sunday,12:00,191.0,132.0,39.0,153.0,132.0,98.0,382.0
5,2023,1,1,Sunday,15:00,256.0,74.0,35.0,205.0,74.0,88.0,366.0
6,2023,1,1,Sunday,18:00,231.0,99.0,28.0,185.0,99.0,70.0,354.0
7,2023,1,2,Monday,07:00,229.0,48.0,38.0,183.0,48.0,95.0,326.0


In [16]:
# List kolom yang akan dikalikan 4
columns_to_multiply = ['MOTOR', 'MOBIL', 'TRUK/BUS', 'SMP_MTR', 'SMP_MBL', 'SMP_TRK', 'SMP']

# Mengalikan nilai kolom dengan 4
df_smp_jam[columns_to_multiply] = df_smp_jam[columns_to_multiply] * 4

In [17]:
df_lhrt = df_smp_jam.copy()

In [18]:
df_lhrt = df_lhrt.drop(columns=['TAHUN','BULAN','TANGGAL'])

In [19]:
df_lhrt.head()

,HARI,JAM,MOTOR,MOBIL,TRUK/BUS,SMP_MTR,SMP_MBL,SMP_TRK,SMP
0,Sunday,08:00,864.0,328.0,176.0,692.0,328.0,440.0,1460.0
1,Sunday,09:00,768.0,452.0,152.0,616.0,452.0,380.0,1448.0
2,Sunday,10:00,808.0,432.0,212.0,648.0,432.0,528.0,1608.0
3,Sunday,11:00,856.0,416.0,144.0,684.0,416.0,356.0,1456.0
4,Sunday,12:00,764.0,528.0,156.0,612.0,528.0,392.0,1528.0


In [20]:
df_lhrt  = df_lhrt.groupby(['HARI', 'JAM'], as_index=False).mean().round()
df_lhrt.head()

,HARI,JAM,MOTOR,MOBIL,TRUK/BUS,SMP_MTR,SMP_MBL,SMP_TRK,SMP
0,Friday,00:00,195.0,259.0,401.0,156.0,259.0,1004.0,1418.0
1,Friday,01:00,169.0,239.0,357.0,135.0,239.0,892.0,1266.0
2,Friday,02:00,151.0,224.0,312.0,122.0,224.0,782.0,1125.0
3,Friday,03:00,132.0,213.0,286.0,106.0,213.0,714.0,1033.0
4,Friday,04:00,119.0,186.0,273.0,95.0,186.0,684.0,966.0


In [21]:
len(df_lhrt)

168

In [22]:
df_lhrt.to_excel('../Data/Preprocessing/Data Losarang/2023/3_lhrt_losarang_2023.xlsx', index=False)